In [ ]:
# python predict_ranking.py --url <신상 옷 URL>

In [1]:
import re
import time
import concurrent.futures
import threading
import pandas as pd
import numpy as np
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def start_chromedriver():
    # ChromeDriver 설정
    service = Service("C:/chromedriver-win64/chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    print("Chrome 드라이버 실행.")
    return service, driver


def end_chromedriver(driver, service):
    # Chrome 드라이버 닫기
    driver.quit()
    print("Chrome 드라이버를 닫음.")
    # service 종료 (필요하면 추가)
    if service:
        service.stop()
    print("ChromeDriver 서비스 종료.")

In [3]:
def get_urls():
    # 무신사 랭킹 페이지 URL
    ranking_urls = {
        "clothes_top": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=001000",
        "outers": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=002000",
        "pants": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=003000",
        "bags": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=004000",
        "skirts": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=100000",
        "fashion_accessories": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=101000",
        "digital_life": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=102000",
        "shoes": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=103000",
        "beauty_items": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=104000",
        "sportswears": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=017000",
        "underwears": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=026000",
        "kids": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=106000"
    }
    return ranking_urls

In [4]:
def get_file_paths(ranking_urls):
    # 현재 날짜 포맷
    current_date = datetime.now().strftime('%Y%m%d')
    
    # URL과 파일 경로 매핑
    file_paths = {
        key: f"data/{key}/{current_date}-ranking-{key.replace('_', '-')}-summary.csv"
        for key in ranking_urls.keys()
    }
    
    return file_paths

In [5]:
# 상품명에 색상있으면 color 컬럼으로 추출
def extract_colors(product_name):
    # 영어 및 한글 색상 리스트
    colors_list = [
        "black", "brown", "blue", "green", "grey", "charcoal", "white", "navy",
        "블랙", "브라운", "블루", "그린", "그레이", "차콜", "화이트", "네이비"
    ]
    # productName에서 색상을 찾고 구분자로 결합
    found_colors = [color for color in colors_list if color in product_name.lower()]
    return " | ".join(found_colors) if found_colors else np.nan

In [6]:
# 숫자+문자 데이터에서 숫자만 출력
def extract_currently_value(text):
    if isinstance(text, float) and np.isnan(text):  # NaN 값 처리
        return 0
    if isinstance(text, list):  # 리스트 처리
        text = text[0] if text else ""  # 리스트가 비어있으면 빈 문자열로 설정
    if not isinstance(text, str):  # 문자열이 아닌 경우 처리
        text = str(text)
    text = text.replace(" ", "")  # 공백 제거
    match = re.search(r"\d+(\.\d+)?", text)  # 숫자 검색
    if not match:  # 숫자가 없을 경우
        return np.nan
    number = float(match.group())  # 숫자 추출
    if "천" in text:  # "천" 단위 처리
        number *= 1000
    return int(number)

In [7]:
def extract_item_data(item):
    """HTML 요소에서 데이터를 추출"""
    try:
        item_id = item.get("data-item-id", np.nan)  # 상품 ID
        product_name_element = item.select_one('p.text-body_13px_reg')  # 상품명
        product_name = product_name_element.text.strip() if product_name_element else np.nan  # 공백 제거
        brand_name_element = item.select_one('p.text-etc_11px_semibold')  # 브랜드명
        brand_name = brand_name_element.text.strip() if brand_name_element else np.nan  # 공백 제거
        price = item.get("data-price", np.nan)  # 가격
        discount_rate = item.get("data-discount-rate", 0)  # 할인율

        # 현재 조회 중
        viewing_span = item.select_one('div.pt-2\\.5 > div > span:nth-child(1)')
        currently_viewing = viewing_span.text.strip() if viewing_span else 0

        # 현재 구매 중
        buying_span = item.select_one('div.pt-2\\.5 > div > span:nth-child(2)')
        currently_buying = buying_span.text.strip() if buying_span else 0

        # "조회 중"만 있고, "구매 중"이 없는 경우
        if viewing_span and not buying_span:
            currently_buying = 0

        # 급상승 여부
        trending_element = item.select_one('div.sc-1m4cyao-1.dYjLwF > div > span > span')
        trending = 1 if trending_element and "급상승" in trending_element.text else 0

        # 랭킹 정보
        ranking_element = item.select_one('div.sc-1m4cyao-1.dYjLwF > span > span')
        ranking = ranking_element.text.strip() if ranking_element else np.nan

        return {
            "productId": item_id,
            "brandName": brand_name,
            "productName": product_name,
            "price": price,
            "discountRate": discount_rate,
            "currentlyViewing": currently_viewing,
            "currentlyBuying": currently_buying,
            "ranking": ranking,
            "trending": trending,
        }
    except Exception as e:
        print(f"데이터 추출 오류: {e}")
        return None

In [8]:
def process_dataframe(data):
    """데이터프레임 생성 및 후처리"""
    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # 데이터프레임 검증
    if "productName" not in df.columns:
        print("데이터프레임에 'productName' 컬럼이 없음.")
        return None

    # 색상 컬럼 추가
    df["colors"] = df["productName"].apply(extract_colors)

    # "현재 조회중", "구매중" 숫자만 추출하여 정수형으로 변환
    df["currentlyViewing"] = df["currentlyViewing"].apply(extract_currently_value)
    df["currentlyBuying"] = df["currentlyBuying"].apply(extract_currently_value)
    
    # 현재 날짜를 모든 row에 추가
    df["date"] = datetime.now().strftime('%Y%m%d')

    return df

In [9]:
def scrape_data(driver, url, file_path):
    """URL에 접근해 데이터를 스크랩하고 CSV로 저장"""
    driver.get(url)
    time.sleep(3)  # 페이지 렌더링 대기

    # HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 데이터 스크래핑
    items = soup.select("div.gtm-view-item-list")
    data = []

    for item in items:
        item_data = extract_item_data(item)
        if item_data:
            data.append(item_data)

    # 데이터가 없을 경우 대비
    if not data:
        print(f"데이터가 비어 있습니다. URL: {url}")
        return

    # 데이터프레임 처리
    df = process_dataframe(data)
    if df is None:
        return

    # CSV 파일 저장
    df.to_csv(file_path, index=False, encoding="utf-8-sig")
    print(f"데이터가 {file_path}에 저장되었습니다.")


In [10]:
def main():
    # ChromeDriver 시작
    service, driver = start_chromedriver()
    
    try:
        # URL과 파일 경로 가져오기
        target_urls = get_urls()
        file_paths = get_file_paths(target_urls)

        # 각 카테고리별 데이터 수집
        for category, url in target_urls.items():
            file_path = file_paths[category]
            print(f"'{category}' 카테고리 데이터를 수집:")
            
            # 데이터를 스크랩하고 CSV로 저장
            scrape_data(driver, url, file_path)
    
    except Exception as e:
        print(f"main 함수 실행 중 오류 발생: {e}")
    
    finally:
        # ChromeDriver 종료
        end_chromedriver(driver, service)


if __name__ == "__main__":
    main()

Chrome 드라이버 실행.
'clothes_top' 카테고리 데이터를 수집:
데이터가 data/clothes_top/20241127-ranking-clothes-top-summary.csv에 저장되었습니다.
'outers' 카테고리 데이터를 수집:
데이터가 data/outers/20241127-ranking-outers-summary.csv에 저장되었습니다.
'pants' 카테고리 데이터를 수집:
데이터가 data/pants/20241127-ranking-pants-summary.csv에 저장되었습니다.
'bags' 카테고리 데이터를 수집:
데이터가 data/bags/20241127-ranking-bags-summary.csv에 저장되었습니다.
'skirts' 카테고리 데이터를 수집:
데이터가 data/skirts/20241127-ranking-skirts-summary.csv에 저장되었습니다.
'fashion_accessories' 카테고리 데이터를 수집:
데이터가 data/fashion_accessories/20241127-ranking-fashion-accessories-summary.csv에 저장되었습니다.
'digital_life' 카테고리 데이터를 수집:
데이터가 data/digital_life/20241127-ranking-digital-life-summary.csv에 저장되었습니다.
'shoes' 카테고리 데이터를 수집:
데이터가 data/shoes/20241127-ranking-shoes-summary.csv에 저장되었습니다.
'beauty_items' 카테고리 데이터를 수집:
데이터가 data/beauty_items/20241127-ranking-beauty-items-summary.csv에 저장되었습니다.
'sportswears' 카테고리 데이터를 수집:
데이터가 data/sportswears/20241127-ranking-sportswears-summary.csv에 저장되었습니다.
'underwears' 카테고리 데이터를 수집:
데이터가